## Читаем данные и визуализируем их

In [3]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib as mpl
mpl.use("TkAgg")

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import mne
import keras

from os import walk


%matplotlib inline
mne.set_log_level('WARNING')
data_path = "./data/resting_state/"

Using TensorFlow backend.


In [4]:
def getBatches(arr, batch_size=50, axis=0):
    n = arr.shape[axis] // batch_size
    return np.array_split(arr, n, axis=axis)

def prepareData(file_path):
    raw = np.array(mne.io.read_raw_brainvision(file_path, preload=True).to_data_frame())
    batches = np.array(getBatches(raw))
    del raw
    
    return batches
#     flat = batches.reshape(-1, batches.shape[1] * batches.shape[2])
#     del batches
    
#     return flat

In [10]:
files = []

for elem in walk(data_path):
    for file in elem[-1]:
        if file[-4:] == "vhdr":
            files.append(file)

data = np.ndarray(shape=(0, 50, 61))
for file in files:
    file_name = data_path + file
    
    batches =  prepareData(file_name)
    data = np.concatenate((data, batches), axis=0)
    print(data.shape)
    

(12057, 50, 61)
(24658, 50, 61)
(31044, 50, 61)
(43123, 50, 61)
(55207, 50, 61)
(61639, 50, 61)
(73696, 50, 61)
(85896, 50, 61)
(97987, 50, 61)
(110042, 50, 61)
(122174, 50, 61)
(134933, 50, 61)
(147012, 50, 61)
(159690, 50, 61)
(171758, 50, 61)
(177798, 50, 61)
(189872, 50, 61)
(202926, 50, 61)
(215004, 50, 61)
(227509, 50, 61)
(233564, 50, 61)
(245606, 50, 61)
(257702, 50, 61)
(270167, 50, 61)
(282311, 50, 61)
(294410, 50, 61)
(307285, 50, 61)
(319349, 50, 61)
(331500, 50, 61)
(343576, 50, 61)
(355908, 50, 61)
(368220, 50, 61)


In [41]:
indices = np.random.permutation(data.shape[0])
training_ids, test_ids = indices[:(data.shape[0] // 10) * 7], indices[(data.shape[0] // 10) * 7:]
X_train, X_test = data[training_ids,:,:], data[test_ids,:,:]

(257754, 50, 61) (110466, 50, 61)


## Создаем нейронную сеть, состоящую из encoder'а и decoder'а, в каждом из которых сделаем по одному полносвязному скрытому слою.

In [42]:
from keras.layers import Input, Dense, Flatten, Reshape
from keras.models import Model

def create_dense_ae():
    encoding_dim = 1000

    # Энкодер
    input_img = Input(shape=(50, 61))
    flat_img = Flatten()(input_img)
    encoded = Dense(encoding_dim, activation="elu")(flat_img)

    # Декодер
    input_encoded = Input(shape=(encoding_dim,))
    flat_decoded = Dense(50*61, activation="elu")(input_encoded)
    decoded = Reshape((50, 61))(flat_decoded)

    # Модели, в конструктор первым аргументом передаются входные слои, а вторым выходные слои
    encoder = Model(input_img, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(input_img, decoder(encoder(input_img)), name="autoencoder")
    return encoder, decoder, autoencoder

In [43]:
encoder, decoder, autoencoder = create_dense_ae()
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 50, 61)            0         
_________________________________________________________________
encoder (Model)              (None, 1000)              3051000   
_________________________________________________________________
decoder (Model)              (None, 50, 61)            3053050   
Total params: 6,104,050
Trainable params: 6,104,050
Non-trainable params: 0
_________________________________________________________________


In [44]:
autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))

Train on 257754 samples, validate on 110466 samples
Epoch 1/50
257754/257754 [==============================] - 126s 490us/step - loss: 172301113.8758 - val_loss: 160282988.2706
Epoch 2/50
257754/257754 [==============================] - 126s 490us/step - loss: 154113621.3583 - val_loss: 152664776.4952
Epoch 3/50
257754/257754 [==============================] - 126s 490us/step - loss: 146879536.3751 - val_loss: 144920509.2932
Epoch 4/50
257754/257754 [==============================] - 126s 488us/step - loss: 144563120.9825 - val_loss: 144801666.0237
Epoch 5/50
257754/257754 [==============================] - 126s 488us/step - loss: 114939322.5141 - val_loss: 109878775.8020
Epoch 6/50
257754/257754 [==============================] - 126s 489us/step - loss: 109552836.1450 - val_loss: 109651969.4344
Epoch 7/50
257754/257754 [==============================] - 126s 487us/step - loss: 109295297.5332 - val_loss: 109475948.8216
Epoch 8/50
257754/257754 [==============================] - 126s 4

In [45]:
encoder.save('encoder_ elu.h5')
decoder.save('decoder_elu.h5')
autoencoder.save('autoencoder_elu.h5')